Here is how to calculate 2 axis gimble angles to point a targeting pod towards a target.

The inputs are the geo-coordinates and the elevation (MSL) of both aircraft and target

In [164]:
import numpy as np
lat1, lon1, elev1 = -35.3632621, 149.1652374, 584+10  # (dynamic)
lat2, lon2, elev2 = -35.35643454149282, 149.16611533539404, 584

Using Haversine fourmula to calcuate target distance, and caluate the bearing to form a 3d vector

In [165]:
# Convert the GPS coordinates from degrees to radians
lat1_rad, lon1_rad = np.deg2rad(lat1), np.deg2rad(lon1)
lat2_rad, lon2_rad = np.deg2rad(lat2), np.deg2rad(lon2)


R = 6371 # Earth radius in kilometers
dlat = lat2_rad - lat1_rad
dlon = lon2_rad - lon1_rad

# Calculate the Haversine distance between the two points
a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R * c

# Calculate the bearing
bearing = np.arctan2(np.sin(dlon) * np.cos(lat2_rad),
                    np.cos(lat1_rad) * np.sin(lat2_rad) -
                    np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(dlon))

# Calculate the x, y and z distances between the two points
x_distance = 1000 * distance * np.cos(bearing)
y_distance = 1000 * distance * np.sin(bearing)
z_distance = elev1 - elev2

Converting the targeting vector from earth (NED) reference frame, to aircraft (Body) reference frame

In [166]:
v_ned = np.array([x_distance, y_distance, z_distance]) 

# Define the rotation matrix that describes the orientation of the body (dynamic)
phi = np.deg2rad(0) # Rotation around x-axis in radians
theta = np.deg2rad(0) # Rotation around y-axis in radians
psi = np.deg2rad(np.degrees(bearing)) # Rotation around z-axis in radians

R_bn = np.array([
[np.cos(theta)*np.cos(psi), np.cos(theta)*np.sin(psi), -np.sin(theta)],
[np.sin(phi)*np.sin(theta)*np.cos(psi)-np.cos(phi)*np.sin(psi), np.sin(phi)*np.sin(theta)*np.sin(psi)+np.cos(phi)*np.cos(psi), np.sin(phi)*np.cos(theta)],
[np.cos(phi)*np.sin(theta)*np.cos(psi)+np.sin(phi)*np.sin(psi), np.cos(phi)*np.sin(theta)*np.sin(psi)-np.sin(phi)*np.cos(psi), np.cos(phi)*np.cos(theta)]
])
# Transform the vector v from the NED frame to the body frame
v_body = np.matmul(R_bn, v_ned)

Calculate the 2 axis gimble angles from the converted vector

In [167]:
# Define the vector v
v = v_body # 3 x 1 matrix

# Compute the direction angles with respect to each axis
Azimuth = np.arctan2(v[1], v[0]) # Azimuth angle
Polar = np.arctan2(v[2], np.sqrt(v[0]**2 + v[1]**2)) # Polar angle

# Convert the angles to degrees
Azimuth = np.rad2deg(Azimuth)
Polar = np.rad2deg(Polar)

print(f'Azimuth:  {np.round(Azimuth,2)}')
print(f'Polar:  {np.round(Polar,2)}')

Azimuth:  -0.0
Polar:  0.75


Negative Polar angles for pointing upward

Positive Azimuth angles for pointing to the right

0 Azimuth angles for pointing forward